In [1]:
import pandas as pd
import great_expectations as ge
import datetime

In [ ]:
# Path file csv yg akan di ubah menjadi dataframe
file_path = 'D:\Hacktiv8\MILESTONE\P2M3\Dataset.csv'

# Input file csv ke dalam dataframe
df = pd.read_csv(file_path)
        
# Tampilkan head atau 5 data pertama dari dataframe
df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [3]:
#Cek tipe data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


In [4]:
# Cek jumlah missing values per kolom
df.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [5]:
df['Item'] = df['Item'].fillna('Unknown')
df['Discount Applied'] = df['Discount Applied'].fillna(False)
df.fillna(0)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,False
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item_23_PAT,38.0,4.0,152.0,Credit Card,In-store,2023-09-03,False
12571,TXN_4009414,CUST_03,Beverages,Item_2_BEV,6.5,9.0,58.5,Cash,Online,2022-08-12,False
12572,TXN_5306010,CUST_11,Butchers,Item_7_BUT,14.0,10.0,140.0,Cash,Online,2024-08-24,False
12573,TXN_5167298,CUST_04,Furniture,Item_7_FUR,14.0,6.0,84.0,Cash,Online,2023-12-30,True


In [6]:
# #Cek Duplikat
df.duplicated().sum()

0

Melihat isi colum categorical yg ada

In [7]:
print("List Category:",df['Category'].unique())
print("List Payment Method:",df['Payment Method'].unique())
print("List Location:",df['Location'].unique())

List Category: ['Patisserie' 'Milk Products' 'Butchers' 'Beverages' 'Food' 'Furniture'
 'Electric household essentials' 'Computers and electric accessories']
List Payment Method: ['Digital Wallet' 'Credit Card' 'Cash']
List Location: ['Online' 'In-store']


# Berdasarkan Eksplorasi, ada beberapa validasi yg bisa dilakukan
1. Transaction ID harus unique
2. Harga product yg dijual tidak boleh negatif (>0)
3. Kategori penjualan hanya mencakup category tertentu
4. Kolom 'Discount Applied' harus berupa boolean
5. Transaction Date harus berupa datetime dengan format YYYY-MM-DD
6. Transaction Date tidak boleh melebihi hari ini
7. Harus ada kolom Transaction Date

In [8]:
from great_expectations.data_context import FileDataContext

# Buat Data Context
context = FileDataContext.create(project_root_dir='./')

In [9]:
# Tambahkan Pandas Datasource
datasource_name = 'csv-data-bintang'
datasource = context.sources.add_pandas(datasource_name)

# Tambahkan Data Asset
asset_name = 'bintang-transaction'
path_to_data = 'P2M3_Bintang-Ksatrio_data_raw.csv'  # Pastikan path benar
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Buat Batch Request
batch_request = asset.build_batch_request()

# Buat Expectation Suite
expectation_suite_name = 'expectation-bintang-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Buat Validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# Cek Sample Data
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [10]:
df_ge = ge.from_pandas(df) 

In [11]:
# 1. Transaction ID harus unique
df_ge.expect_column_values_to_be_unique("Transaction ID")

{
  "success": true,
  "result": {
    "element_count": 12575,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [12]:
print("Nilai Max:", df["Price Per Unit"].max())
print("Nilai Min:", df['Price Per Unit'].min())

Nilai Max: 41.0
Nilai Min: 5.0


In [ ]:
# 2. Harga product yg dijual tidak boleh negatif (>0)
df_ge.expect_column_values_to_be_between("Price Per Unit", min_value=0)

{
  "success": true,
  "result": {
    "element_count": 12575,
    "missing_count": 609,
    "missing_percent": 4.842942345924453,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
# 3. Kategori penjualan hanya ada:
# Patisserie,Milk Products,Butchers,Beverages,Food,Furniture,
# Electric household essentials, dan Computers and electric accessories
valid_categories = ['Patisserie','Milk Products','Butchers','Beverages',
                    'Food','Furniture','Electric household essentials',
                    'Computers and electric accessories']
df_ge.expect_column_values_to_be_in_set("Category", valid_categories)

{
  "success": true,
  "result": {
    "element_count": 12575,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
# 4. Discount Applied harus berupa boolean
df_ge.expect_column_values_to_be_in_type_list(
    "Discount Applied", ["bool"])

{
  "success": true,
  "result": {
    "observed_value": "bool_"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
# 5. Transaction Date harus berupa datetime 
# dengan format YYYY-MM-DD 
df_ge.expect_column_values_to_match_strftime_format(
    "Transaction Date", "%Y-%m-%d")

{
  "success": true,
  "result": {
    "element_count": 12575,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
# 6. Transaction Date tidak boleh melebihi hari ini
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
df_ge.expect_column_values_to_be_between(
    "Transaction Date", max_value=today_date)

{
  "success": true,
  "result": {
    "element_count": 12575,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
# 7. Harus ada kolom Transaction Date di data
df_ge.expect_column_to_exist(
    column="Transaction Date"
)

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [19]:
validator.save_expectation_suite(discard_failed_expectations=False)